In [1]:
import pickle
from collections import Counter
from datetime import datetime
from pathlib import Path

import joblib
import keras
import matplotlib.pyplot as plt
import polars as pl
import seaborn as sns
import numpy as np
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)

from utils import *

In [2]:
# Set up the configuration for the polars library
cfg = pl.Config()
cfg.set_tbl_rows(100)
cfg.set_tbl_width_chars(200)
cfg.set_fmt_str_lengths(200)

pc = ProjectConfig()
BASE_TRAINING_MODEL_DIR = pc.project_root_dir.joinpath("model_selection").joinpath(
    "model_saves_fc"
)
FINE_TUNED_MODEL_DIR = pc.project_root_dir.joinpath("model_selection").joinpath(
    "model_fine_tuned_saves_fc"
)
MODEL_NAMES = ["EfficientNetB7", "EfficientNetV2L", "EfficientNetV2M", "VGG19"]
FINE_TUNED_MODEL_PATHS = []
CLASS_LIST_PATHS = []
BASE_TRAINING_TIME_PATHS = []
FINE_TUNED_TRAINING_TIME_PATHS = []
for model_name in MODEL_NAMES:
    FINE_TUNED_MODEL_PATHS.append(FINE_TUNED_MODEL_DIR.joinpath(model_name + ".h5"))
    CLASS_LIST_PATHS.append(
        pc.project_root_dir.joinpath("model_selection").joinpath(
            f"class_list_{model_name}.lzma"
        )
    )
    BASE_TRAINING_TIME_PATHS.append(
        BASE_TRAINING_MODEL_DIR.joinpath(f"{model_name}_TRAINING_TIME.lzma")
    )
    FINE_TUNED_TRAINING_TIME_PATHS.append(
        FINE_TUNED_MODEL_DIR.joinpath(f"{model_name}_TRAINING_TIME.lzma")
    )

No configuration file found. Using defaults.


In [3]:
TEST_DATA = pc.data_root_dir.joinpath("FruitsClassification/Fruits Classification/test")
if TEST_DATA.exists() and TEST_DATA.is_dir():
    print("Test data found, loading")
    test_data = tf.keras.utils.image_dataset_from_directory(
        TEST_DATA,
        labels="inferred",
        label_mode="int",
        batch_size=32,
        image_size=(224, 224),
        shuffle=False,
    )
else:
    print("No test data found, where did it go?")

Test data found, loading
Found 100 files belonging to 5 classes.


In [4]:
results = []

for i, model_path in enumerate(FINE_TUNED_MODEL_PATHS):
    if model_path.exists():
        print(f"Model {model_path} found, loading")
        tf_model = tf.keras.models.load_model(model_path)
        tf_model.trainable = False
        print(f"Model {model_path} loaded")
    else:
        print(f"Model not found at path: {model_path}")
    if CLASS_LIST_PATHS[i].exists():
        print(f"Class list found at path: {CLASS_LIST_PATHS[i]}, loading")
        class_list = joblib.load(CLASS_LIST_PATHS[i])
        print(f"Class list loaded")

    start_predict = datetime.now()
    predictions = tf_model.predict(test_data)
    end_predict = datetime.now()
    prediction_time = end_predict - start_predict
    predicted_classes = np.argmax(predictions, axis=-1)
    true_labels = []
    for images, true_label in test_data:
        true_labels.extend(true_label)
    all_predictions = np.array(predicted_classes)
    all_true_labels = np.array(true_labels)
    precision = precision_score(all_true_labels, all_predictions, average="weighted")
    recall = recall_score(all_true_labels, all_predictions, average="weighted")
    accuracy = accuracy_score(all_true_labels, all_predictions)
    f1 = f1_score(all_true_labels, all_predictions, average="weighted")
    base_training_time = joblib.load(BASE_TRAINING_TIME_PATHS[i])
    fine_tuned_training_time = joblib.load(FINE_TUNED_TRAINING_TIME_PATHS[i])
    results.append(
        {
            "model": MODEL_NAMES[i],
            "base_training_time": base_training_time,
            "fine_tuned_training_time": fine_tuned_training_time,
            "total_training_time": base_training_time + fine_tuned_training_time,
            "prediction_time": prediction_time,
            "precision": precision,
            "recall": recall,
            "accuracy": accuracy,
            "f1": f1,
        }
    )
    print(f"Deleting Model {tf_model}")
    del tf_model
    tf.keras.backend.clear_session()
results_df = pl.DataFrame(results)
results_df

Model C:\Users\teddy\Documents\01-Berkeley\210\gotmeals\model_selection\model_fine_tuned_saves_fc\EfficientNetB7.h5 found, loading


Model C:\Users\teddy\Documents\01-Berkeley\210\gotmeals\model_selection\model_fine_tuned_saves_fc\EfficientNetB7.h5 loaded
Class list found at path: C:\Users\teddy\Documents\01-Berkeley\210\gotmeals\model_selection\class_list_EfficientNetB7.lzma, loading
Class list loaded
4/4 [==============================] - 12s 2s/step
Deleting Model <keras.src.engine.functional.Functional object at 0x000001876F88EE10>
Model C:\Users\teddy\Documents\01-Berkeley\210\gotmeals\model_selection\model_fine_tuned_saves_fc\EfficientNetV2L.h5 found, loading
Model C:\Users\teddy\Documents\01-Berkeley\210\gotmeals\model_selection\model_fine_tuned_saves_fc\EfficientNetV2L.h5 loaded
Class list found at path: C:\Users\teddy\Documents\01-Berkeley\210\gotmeals\model_selection\class_list_EfficientNetV2L.lzma, loading
Class list loaded
4/4 [==============================] - 13s 2s/step


model,base_training_time,fine_tuned_training_time,total_training_time,prediction_time,precision,recall,accuracy,f1
str,duration[μs],duration[μs],duration[μs],duration[μs],f64,f64,f64,f64
"""EfficientNetB7""",15m 15s 602087µs,23m 42s 482739µs,38m 58s 84826µs,12s 207872µs,0.954113,0.95,0.95,0.948724
"""EfficientNetV2L""",26m 3s 256165µs,26m 7s 905710µs,52m 11s 161875µs,12s 986258µs,0.954865,0.95,0.95,0.94919
"""EfficientNetV2M""",10m 8s 696439µs,18m 53s 519488µs,29m 2s 215927µs,7s 628715µs,0.950707,0.95,0.95,0.949424
"""VGG19""",27m 35s 899679µs,8m 41s 139304µs,36m 17s 38983µs,5s 247271µs,0.772131,0.76,0.76,0.748899
